In [6]:
import function as fu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import tensorflow as tf
import random

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, roc_auc_score, roc_curve, f1_score, precision_score, recall_score, matthews_corrcoef


from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv('data/creditcard.csv')
data.rename(columns= {'LIMIT_BAL':'given_credit',
                       'PAY_0':'status_apr',
                       'PAY_2':'status_may',
                       'PAY_3':'status_jun',
                       'PAY_4':'status_jul',
                       'PAY_5':'status_aug',
                       'PAY_6':'status_sep',

                       'BILL_AMT1':'bill_apr',
                       'BILL_AMT2':'bill_may',
                       'BILL_AMT3':'bill_jun',
                       'BILL_AMT4':'bill_jul',
                       'BILL_AMT5':'bill_aug',
                       'BILL_AMT6':'bill_sep',
                       
                       'PAY_AMT1':'pay_apr',
                       'PAY_AMT2':'pay_may',
                       'PAY_AMT3':'pay_jun',
                       'PAY_AMT4':'pay_jul',
                       'PAY_AMT5':'pay_aug',
                       'PAY_AMT6':'pay_sep',
                       },
                        
            inplace = True)

# As a syntaxis rule, the rest of the column names are changed to lower case
data.columns = data.columns.str.lower()

#Blank spaces are also replaced by "_"
data.columns = data.columns.str.replace(" ", "_")

#Our tatget variable has a very long name, so it is shortened to "default_payment"
data.rename(columns= {'default.payment.next.month':'default_payment'}, inplace = True)

marriage = pd.DataFrame()

#Frequency table by marriage value
marriage["qty_data"] = pd.DataFrame(data.marriage.value_counts())

#Mean age by marriage value:
marriage["mean_age"] = pd.DataFrame(round(data.groupby("marriage")["age"].mean(),2))

#Mean default_payment
marriage["deafult_rate"] = pd.DataFrame(round(data.groupby("marriage")["default_payment"].mean(),2))

data_clean = data
data_clean.loc[(data_clean['marriage']==0),'marriage']=3

education = pd.DataFrame()

#Frequency table by marriage value
education["qty_data"] = pd.DataFrame(data.education.value_counts())

#Mean age by marriage value:
education["mean_age"] = pd.DataFrame(round(data.groupby("education")["age"].mean(),2))

#Mean default_payment
education["deafult_rate"] = pd.DataFrame(round(data.groupby("education")["default_payment"].mean(),2))

data_clean.loc[(data_clean['education']==0) | (data_clean['education']==5) | (data_clean['education']==6),
               'education']=4

status = pd.DataFrame()

#Frequency table by marriage value
status["qty_data"] = pd.DataFrame(data.status_apr.value_counts())

#Mean age by marriage value:
status["mean_age"] = pd.DataFrame(round(data.groupby("status_apr")["age"].mean(),2))

#Mean default_payment
status["deafult_rate"] = pd.DataFrame(round(data.groupby("status_apr")["default_payment"].mean(),2))

data_FE = data_clean
#Set "education" as catigory type
data_FE["education"] = data_FE ["education"].astype("category")

#Creating dummy features
edu_dummies = pd.get_dummies(data_FE['education'], prefix='edu')

#Adding these new dummy variables to our dataset
data_FE = pd.concat( [data_FE, edu_dummies] , axis = 1)
data_FE = data_FE.drop(columns=['education']);

data_FE.columns = data_FE.columns.str.replace(",", "-") # subst. "," for "-"
data_FE.columns = data_FE.columns.str.replace(" ", "")  # deleting spaces
data_FE.columns = data_FE.columns.str.replace("(", "")  # deleting "("
data_FE.columns = data_FE.columns.str.replace(")", "")  # deleting ")"
data_FE.columns = data_FE.columns.str.replace("]", ""); # deleting "]"

data_premod = data_FE
# Separating into input data and target variables

#Eliminating the target column from the input data
X_original =  pd.DataFrame( data_premod.drop(columns='default_payment'))

# Target varibale as independent dataframe
Y_original = pd.DataFrame( data_premod["default_payment"] )

# Standard Scaler from SciKit Learn applies the Z-score normalizaion to the input "X_original"


scaler = StandardScaler()

X_scaled = pd.DataFrame( scaler.fit_transform(X_original) )
X_scaled.columns = X_original.columns;
#X_scaled.iloc[:,1:13].describe()

Y_scaled = Y_original;
data_scaled= pd.concat([X_scaled, Y_scaled], axis=1, join='inner');
    
formatted_data_scaled = pd.DataFrame()
formatted_data_scaled =  pd.melt(data_scaled, ["default_payment"], var_name="feature");

X_train, X_test, y_train, y_test= train_test_split(X_scaled, Y_scaled,test_size=0.2,shuffle = True,random_state = 42)

In [9]:
%%time
c_01 = 3
c_10 = 1
p_list = [2,4,6,8,10]
N_list = [[0.2, 0.5], [0.9,1.1], [1.5, 1.8]]
N_list1 = [ [0.1, 0.8], [0.2,0.5] , [0.6, 1.1]  , [0.7, 1] , [0.8, 1.2], [0.8, 1.4], [0.9,1.4] , [1, 1.1],  [1, 1.2] , [1.1,1.3], [1.5, 1.8] ]
for N1_ratio,N2_ratio in N_list1:
    N1= int(np.unique(y_test, return_counts = True)[1][1] * N1_ratio)
    N2= int(np.unique(y_test, return_counts = True)[1][1] * N2_ratio)
        
    list_con_mat = []
    list_p = []
    list_cost = []
    list_number = []
    list_precision = []
    list_recall = []
    list_f1 = []
    list_auc = []
    list_mcc = []
    list_gmean = []

    for p in p_list:
        ticket = True
        while ticket:
            con_mat,cost, number, precision, recall, f1, auc,mcc, gmean = algorithm_spo(c_01,p, N1,N2)
            if not (con_mat[0] == N1) or (con_mat[-1] == N2): ticket = False

        #print(f'initial_p:{p},cost:{cost},precision:{precision},recall:{recall},f1:{f1}')
        list_p.append(p)
        list_cost.append(cost)
        list_number.append(number)
        list_precision.append(precision)
        list_recall.append(recall)
        list_f1.append(f1)
        list_con_mat.append(con_mat)
        list_auc.append(auc)
        list_mcc.append(mcc)
        list_gmean.append(gmean)


    dict = {'p': list_p, 'num1':list_number, 'con_mat':list_con_mat, 'cost': list_cost, 'list_precision':list_precision, 'list_recall':list_recall, 'list_f1':list_f1,
           'list_auc': list_auc, 'list_mcc': list_mcc, 'list_gmean': list_gmean}   

    df = pd.DataFrame(dict) 
    method = 'spo'
    # saving the dataframe 
    df.to_csv('result/case2/result_{}_{}_{}.csv'.format(method,N1,N2))

CPU times: user 30min 48s, sys: 3min 10s, total: 33min 58s
Wall time: 23min 58s


In [8]:
def spo_plus_loss(c_01):
    c_01 = tf.cast(c_01, tf.float64)
    
    def my_loss(y_true,y_pred):
        #y_pred = tf.clip_by_value(y_pred, 0.0001, 1 - 0.0001)
        y_pred = tf.cast(y_pred, tf.float64)
        y_true = tf.cast(y_true, tf.float64)
        
        max_1 = tf.cast(tf.math.maximum(2 * y_pred - 1,0),tf.float64)
        max_2 = tf.cast(tf.math.maximum(1-2 * y_pred, 0), tf.float64)
        
        loss =  y_pred*(1-y_pred)+ max_1 * c_01 * y_true + (1- y_true) * max_2 + c_01 * y_true * (1-2 * y_pred) + (1 - y_true) * (2*y_pred - 1)
        #loss = tf.cast(y_true, tf.float64) * y_pred +(1-tf.cast(y_true, tf.float64)) * ( 1-y_pred)
         
        #if max_1 ==0: max_1 =tf.cast(1, tf.float64)
        #if max_2 ==0: max_2 =tf.cast(1, tf.float64)
            
        #loss = tf.math.multiply_no_nan(x= tf.math.log(max_1), y= tf.cast(y_true, tf.float64))* c_01 +  tf.math.multiply_no_nan( x=tf.math.log(max_2), y= (1- tf.cast(y_true, tf.float64)) )*c_01

        #loss = ((tf.cast(y_true, tf.float64) * (1-y_pred))*c_01 + ((1-tf.cast(y_true, tf.float64)) * y_pred ))
                                                      
        return loss
    return my_loss

def Model_spo(c_01,X_train, y_train, X_test, y_test):
    
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape = (np.array(X_train).shape[-1],)),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss= spo_plus_loss(c_01))
    
    model.fit(X_train,y_train,shuffle = True, epochs= 30,verbose =0)
    y_pred = model.predict(X_test, batch_size=16, verbose =0)
 
    return y_pred

def algorithm_spo(c_01,p, N1,N2):
        
    y_pred = Model_spo(c_01,X_train, y_train, X_test, y_test)
    fpr, fnr, thresholds = fu.fpr_fnr(y_pred, y_test)

    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]
    
    if D_p <= N1:
        num = N1
    elif N2 <= D_p:
        num = N2
    elif N1 < D_p < N2:
        num = D_p
    
    th = sorted(y_pred, reverse = True)[num]
    y_result = np.where(y_pred>th, 1, 0)
    TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()

    if (FP+TP) < N1:
        ticket1 = False
        ticket2 = False
        count = 0
        
        target = N1 - (FP+TP) 
        for i in range(len(y_result)):
            if y_result[i] ==0:
                y_result[i] =1
                count +=1
            if count == target:
                ticket1 = True

            if ticket1 == True:
                break

    elif N2 < (FP+TP):
        ticket1 = False
        ticket2 = False
        count = 0
        
        target = (FP+TP) - N2
        for i in range(len(y_result)):
            if y_result[i] ==1:
                y_result[i] =0
                count +=1
            if count == target:
                ticket2 = True

            if ticket2 == True:
                break

    TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()
    cost = confusion_matrix(y_test, y_result).T[0][1] * c_01 + confusion_matrix(y_test, y_result).T[1][0] * c_10
    cost = round(cost)
 
    number = confusion_matrix(y_test, y_result).T[1][1] + confusion_matrix(y_test, y_result).T[1][0]       
    p = round(precision_score(y_test, y_result),2)
    r = round(recall_score(y_test, y_result),2)
    f = round(f1_score(y_test, y_result),2)
    auc = round(roc_auc_score(y_test, y_result),2)
    mcc = round(matthews_corrcoef(y_test, y_result),2)
    
    tpr = TP / (TP+FN)
    tnr = TN / (TN+FP)
    gmean = np.sqrt(tpr * tnr)
    
    con_mat = confusion_matrix(y_test, y_result).T.flatten().tolist()
    
    return con_mat,cost, number, p, r, f, auc,mcc, gmean